In [2]:
import os
import sys
import torch
import pandas as pd
from tqdm.notebook import tqdm
import pickle as pk
import numpy as np

device = torch.cuda.set_device(0)

BASE_DIR = ".."
MODEL_BASE_DIR = f"{BASE_DIR}/best_models"
MODEL_DIR = f"{BASE_DIR}/models"
DATA_DIR = f"{BASE_DIR}/nbdata"
os.makedirs(DATA_DIR,exist_ok=True)
sys.path.append(BASE_DIR)

In [3]:
from transformers import T5Tokenizer, T5EncoderModel, pipeline
import re

In [4]:
tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_uniref50', do_lower_case=False,cache_dir=MODEL_DIR)
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50",cache_dir=MODEL_DIR)

Some weights of the model checkpoint at Rostlab/prot_t5_xl_uniref50 were not used when initializing T5EncoderModel: ['decoder.embed_tokens.weight', 'decoder.block.0.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.0.SelfAttention.k.weight', 'decoder.block.0.layer.0.SelfAttention.v.weight', 'decoder.block.0.layer.0.SelfAttention.o.weight', 'decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.0.layer.0.layer_norm.weight', 'decoder.block.0.layer.1.EncDecAttention.q.weight', 'decoder.block.0.layer.1.EncDecAttention.k.weight', 'decoder.block.0.layer.1.EncDecAttention.v.weight', 'decoder.block.0.layer.1.EncDecAttention.o.weight', 'decoder.block.0.layer.1.layer_norm.weight', 'decoder.block.0.layer.2.DenseReluDense.wi.weight', 'decoder.block.0.layer.2.DenseReluDense.wo.weight', 'decoder.block.0.layer.2.layer_norm.weight', 'decoder.block.1.layer.0.SelfAttention.q.weight', 'decoder.block.1.layer.0.SelfAttention.k.weight', 'decoder.block.1.layer.0.SelfAtte

In [5]:
model = model.cuda().eval()

In [24]:
s = 'MAEKQKHDGRVKIGHYVLGDTLGVGTFGKVKIGEHQLTGHKVAVKILNRQKIRSLDVVGKIKREIQNLKLFRHPHIIKLYQVISTPTDFFMVMEYVSGGELFDYICKHGRVEEMEARRLFQQILSAVDYCHRHMVVHRDLKPENVLLDAHMNAKIADFGLSNMMSDGEFLRTSCGSPNYAAPEVISGRLYAGPEVDIWSCGVILYALLCGTLPFDDEHVPTLFKKIRGGVFYIPEYLNRSVATLLMHMLQVDPLKRATIKDIREHEWFKQDLPSYLFPEDPSYDANVIDDEAVKEVCEKFECTESEVMNSLYSGDPQDQLAVAYHLIIDNRRIMNQASEFYLASSPPSGSFMDDSAMHIPPGLKPHPERMPPLIADSPKARCPLDALNTTKPKSLAVKKAKWHLGIRSQSKPYDIMAEVYRAMKQLDFEWKVVNAYHLRVRRKNPVTGNYVKMSLQLYLVDNRSYLLDFKSIDDEVVEQRSGSSTPQRSCSAAGLHRPRSSFDSTTAESHSLSGSLTGSLTGSTLSSVSPRLGSHTMDFFEMCASLITTLAR'

In [25]:
seq = [s]

In [7]:
def space_sequence(x):
    return " ".join(list(x))

In [8]:
seq = [space_sequence(seq[0])]

In [9]:
ids = tokenizer.batch_encode_plus(seq, add_special_tokens=True, padding=True)

input_ids = torch.tensor(ids['input_ids'])
attention_mask = torch.tensor(ids['attention_mask'])

In [10]:
input_ids = input_ids.cuda()
attention_mask = attention_mask.cuda()

In [31]:
with torch.no_grad():
    embedding = model(input_ids=input_ids,attention_mask=attention_mask).last_hidden_state

In [32]:
seq_len = len(s)
start_Idx = 1
end_Idx = seq_len+1
seq_emb = embedding.squeeze()[start_Idx:end_Idx]

In [33]:
seq_emb

tensor([[ 0.1894, -0.1595,  0.2575,  ...,  0.1354,  0.1088,  0.2647],
        [ 0.0953, -0.1006,  0.1352,  ...,  0.1090,  0.0385,  0.3230],
        [ 0.0251, -0.1949,  0.1189,  ..., -0.0659,  0.3112,  0.2964],
        ...,
        [-0.1379, -0.1265,  0.2116,  ..., -0.0962,  0.0582,  0.0811],
        [-0.1329, -0.1196,  0.0572,  ..., -0.0551, -0.0745, -0.0734],
        [ 0.0202, -0.0432,  0.0892,  ..., -0.0503,  0.0445,  0.0005]],
       device='cuda:0')

In [30]:
seq_emb

tensor([[ 0.1894, -0.1595,  0.2575,  ...,  0.1354,  0.1088,  0.2647],
        [ 0.0953, -0.1006,  0.1352,  ...,  0.1090,  0.0385,  0.3230],
        [ 0.0251, -0.1949,  0.1189,  ..., -0.0659,  0.3112,  0.2964],
        ...,
        [-0.1379, -0.1265,  0.2116,  ..., -0.0962,  0.0582,  0.0811],
        [-0.1329, -0.1196,  0.0572,  ..., -0.0551, -0.0745, -0.0734],
        [ 0.0202, -0.0432,  0.0892,  ..., -0.0503,  0.0445,  0.0005]],
       device='cuda:0')

In [34]:
seq_len = (attention_mask[0] == 1).sum()
seq_emd = embedding[0][:seq_len-1]

In [35]:
seq_emd

tensor([[ 0.1591, -0.0538,  0.0769,  ..., -0.0248, -0.0416,  0.1548],
        [ 0.1894, -0.1595,  0.2575,  ...,  0.1354,  0.1088,  0.2647],
        [ 0.0953, -0.1006,  0.1352,  ...,  0.1090,  0.0385,  0.3230],
        ...,
        [-0.1193, -0.0576,  0.0915,  ..., -0.0994,  0.1092,  0.1397],
        [-0.1379, -0.1265,  0.2116,  ..., -0.0962,  0.0582,  0.0811],
        [-0.1329, -0.1196,  0.0572,  ..., -0.0551, -0.0745, -0.0734]],
       device='cuda:0')

In [29]:
seq_emd

tensor([[ 0.1591, -0.0538,  0.0769,  ..., -0.0248, -0.0416,  0.1548],
        [ 0.1894, -0.1595,  0.2575,  ...,  0.1354,  0.1088,  0.2647],
        [ 0.0953, -0.1006,  0.1352,  ...,  0.1090,  0.0385,  0.3230],
        ...,
        [-0.1193, -0.0576,  0.0915,  ..., -0.0994,  0.1092,  0.1397],
        [-0.1379, -0.1265,  0.2116,  ..., -0.0962,  0.0582,  0.0811],
        [-0.1329, -0.1196,  0.0572,  ..., -0.0551, -0.0745, -0.0734]],
       device='cuda:0')

In [28]:
seq_emd.shape

torch.Size([552, 1024])

In [15]:
embedding.shape

torch.Size([1, 553, 1024])

In [16]:
len(seq[0])

1103

In [2]:
from tdc import utils
from tdc.benchmark_group import dti_dg_group

names = utils.retrieve_benchmark_names('DTI_DG_Group')
group = dti_dg_group(path = '../nbdata')
benchmark = group.get('bindingdb_patent')
name = benchmark['name']
train_val, test = benchmark['train_val'], benchmark['test'] # Natural log transformed (kd/ki/ic50??)


Found local copy...


In [ ]:
|